In [1]:
import pandas as pd 
import numpy as np  
import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns
import datetime
from datetime import date, timedelta

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from scipy.stats import norm
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix

import pylab as pl
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import pickle

from sklearn.tree import DecisionTreeClassifier

from sklearn.tree import export_graphviz 
    #In the cmd line type: conda install python-graphviz
import graphviz
import os
os.environ["PATH"] += os.pathsep + 'D:/Program Files (x86)/Graphviz2.38/bin/'

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import roc_curve
from sklearn.ensemble import RandomForestClassifier
import array as arr

C:\Users\oripo\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
df = pd.read_csv("no_show_original_data_2016_with_update_weather.csv") 
df=df.drop(['Unnamed: 0'], axis=1)

In [3]:
#ScheduledDay קיצוץ התאריך של והופך את התאריך לסוג דייטיים
df['ScheduledDay'] =df['ScheduledDay'].apply(str)
df['ScheduledDay'] = df['ScheduledDay'].str.slice(0, 10, 1) 
df['ScheduledDay'] =  pd.to_datetime(df['ScheduledDay'], format='%Y-%m-%d')

In [4]:
# הפיכת שדות מז"א למספריים
df['prcp']=df['prcp'].astype(float).astype(int)
df['temp']=df['temp'].astype(float).astype(int)

In [5]:
#AppointmentDay הופך את התאריך לסוג דייטיים
df['AppointmentDay'] =df['AppointmentDay'].apply(str)
df['AppointmentDay'] = df['AppointmentDay'].str.slice(0, 10, 1) 
df['AppointmentDay'] =  pd.to_datetime(df['AppointmentDay'], format='%Y-%m-%d')

In [6]:
# הכנסת קודים לשכונות
df.loc[:, 'Neighbourhood_codes'] = df["Neighbourhood"].astype("category").cat.codes+1
# הפיכת שדות קטגוריים לבינריים
df.loc[:, 'Gender_'] = df["Gender"].astype("category").cat.codes
#Woman=0, Man=1
df.loc[:, 'No-show_'] = df["No-show"].astype("category").cat.codes
#show=0, no-show=1 

In [7]:
#הוספת שדה של יום בשבוע של הבדיקה
dayOfWeek={0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
df['Appointment_Week_Day'] = df['AppointmentDay'].dt.dayofweek.map(dayOfWeek)

In [8]:
#הוספת שדה של המרווח בין הבדיקות
df['Delta_Dates'] = (df['AppointmentDay']-df['ScheduledDay']).dt.days
# בדיקות שתואמו אחרי הבדיקה עצמה יופיעו כמינוס 1-כי לא הגיוני
mask = df.Delta_Dates < 0
column_name = 'Delta_Dates'
df.loc[mask, column_name] = -1

In [9]:
# חיתוך כל מי שיש לו דלתא קטנה מ1
#df[df.Delta_Dates < 1] מוחק
#38562 רשומות של פגישות שנקבעו באותו תאריך
# 5 רשומות של דלתא שלילית=לא הגיוני
df = df.drop(df[df.Delta_Dates < 1].index)

In [10]:
#Get Dummies לימי השבוע של הפגישה
week_day=pd.get_dummies(df["Appointment_Week_Day"])
week_day["AppointmentID"] = df["AppointmentID"]
df=df.merge(week_day, how='left', on='AppointmentID')
df = df.drop(['Appointment_Week_Day'], axis=1)

In [11]:
df["yesterday"] = df["AppointmentDay"] - timedelta(1)

In [12]:
df['Appointments_untill_yesterday'] = '0'
df['no_show_untill_yesterday'] = '0'

In [13]:
df['no_show_freq_for_id'] = '0'

In [14]:
for i, row in df.iterrows():
    yesterday_date=df.iloc[i]["yesterday"]
    this_id=df.iloc[i]["PatientId"]
    #חיתוך לדאטהפריים של פעם עד אתמול
    df_yesterday=df[df["AppointmentDay"] <= yesterday_date] 
    #חיתוך רק לבעל הIDs
    df_yesterday=df_yesterday.loc[df['PatientId'] == this_id]
    #סכימת כמה פעמים היו לו פגישות
    countt=df_yesterday['No-show_'].count()
    summ= df_yesterday['No-show_'].sum()
    if countt == 0: 
        df.set_value(i,"Appointments_untill_yesterday",0, takeable=False)
        df.set_value(i,"no_show_untill_yesterday", 0, takeable=False)
    else:
        df.set_value(i,"Appointments_untill_yesterday",countt, takeable=False)
        df.set_value(i,"no_show_untill_yesterday", summ, takeable=False)

C:\Users\oripo\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if sys.path[0] == '':
C:\Users\oripo\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]
C:\Users\oripo\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app
C:\Users\oripo\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()


In [15]:
for i, row in df.iterrows():
    totall=df.iloc[i]["Appointments_untill_yesterday"]
    no_show_count=df.iloc[i]["no_show_untill_yesterday"]
    if totall > 0:
        a=no_show_count/totall
        df.set_value(i,"no_show_freq_for_id",a, takeable=False)

C:\Users\oripo\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [16]:
df = df.sort_values(by='AppointmentDay')

In [102]:
#Get Dummies לשכונות
NC=pd.get_dummies(df["Neighbourhood_codes"])
NC["AppointmentID"] = df["AppointmentID"]
df=df.merge(NC, how='left', on='AppointmentID')

KeyError: 'Neighbourhood_codes'

In [19]:
df.columns

Index(['Age', 'Scholarship', 'Hipertension', 'Diabetes', 'Alcoholism',
       'Handcap', 'SMS_received', 'ScheduledWeekDay', 'prcp', 'temp',
       'Gender_', 'No-show_', 'Delta_Dates', 'Friday', 'Monday', 'Saturday',
       'Thursday', 'Tuesday', 'Wednesday', 'Appointments_untill_yesterday',
       'no_show_untill_yesterday', 'no_show_freq_for_id'],
      dtype='object')

In [18]:
#הורדת שדות מיותרים
df = df.drop(columns=['PatientId', 'AppointmentID', 'Gender', 'ScheduledDay', 'No-show', 'Neighbourhood', 'Neighbourhood_codes','AppointmentDay','yesterday'], axis = 1)

In [ ]:
#new_index_order = ['Age','Gender_','Delta_Dates','SMS_received', 'Friday', 'Monday', 'Saturday', 'Thursday', 'Tuesday', 'Wednesday','Scholarship', 'Hipertension', 'Diabetes', 'Alcoholism',
   #    'Handcap','No-show_']
#df = df.reindex(columns=new_index_order)

In [20]:
pickle.dump(df, open("df_data_a.dat", "wb"))